# 1. Generate images set out of original lunar DEM tile (60 m/pix)
download file and initialize workspace: 

In [ ]:
# !wget -P ../data/ http://pds-geosciences.wustl.edu/lro/lro-l-lola-3-rdr-v1/lrolol_1xxx/data/sldem2015/tiles/jp2/sldem2015_512_00n_30n_000_045.jp2
from PIL import Image
import time
Image.MAX_IMAGE_PIXELS = 353894500
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import input_data_gen as igen
import utils.processing as proc
import h5py
import utils.maplot

data_path='../data/'

In [ ]:
# Read source image.
img = Image.open(data_path + "/sldem2015_512_00n_30n_000_045.jp2")
print(img.format, img.size, img.mode)
# [Min long, max long, min lat, max lat] dimensions of source image.
source_cdim = [0., 45., 0., 30.]

In [ ]:
start_time = time.time()
fig = plt.figure(figsize=[6, 3])
dem_map = plt.imshow(img, extent=source_cdim, aspect="equal", cmap="YlGn")
elapsed_time = time.time() - start_time
print(elapsed_time)

Crop img to img_cr:

In [ ]:
box = (0,1800,3600,3600)
ex_box = source_cdim*(np.array(box)/(img.size+img.size))[[0,2,1,3]]
img_cr=img.crop(box)
print(img_cr.format, img_cr.size, img_cr.mode)

Conversion from 16-bit to 8-bit:

In [ ]:
start_time = time.time()

img16=np.asarray(img_cr)
img16vec=np.concatenate(img16)

fig=plt.figure(figsize=(12,5))
#plot original image 16 bit+histogram:
plt.subplot(2,2,1)
plt.title('original 16 bit')
plt.imshow(img16, extent=source_cdim, aspect="equal", cmap="YlGn")
plt.colorbar()

plt.subplot(2,2,3)
plt.hist(img16vec, 256*2);


#transformation: 
min_val = np.min(img16vec)
dif = (np.max(img16vec)-min_val)
img8 = np.uint8((img16-min_val)/dif*256)

#plot transformed 8 bit+hist:
plt.subplot(2,2,2)
plt.title('transformed 8 bit')
plt.imshow(img8, extent=source_cdim, aspect="equal", cmap="YlGn")
plt.colorbar()

plt.subplot(2,2,4)
plt.hist(np.concatenate(img8), 256*2);

elapsed_time = time.time() - start_time
print(elapsed_time)

Define conversion function:

In [ ]:
from PIL import Image
import numpy as np
def convert16to8bit(img):
    """Transform PIL image of 16-bit to 8-bit"""
    img16=np.asarray(img)
    img16vec=np.concatenate(img16)

    #transformation: 
    min_val = np.min(img16vec)
    dif = (np.max(img16vec)-min_val)
    img8 = np.uint8((img16-min_val)/dif*256)

    return Image.fromarray(img8)

In [ ]:
print(img.format, img.size, img.mode)
start_time = time.time()
con_img=convert16to8bit(img)
elapsed_time = time.time() - start_time
print(elapsed_time)
print(con_img.format, con_img.size, con_img.mode)

In [ ]:
# Read the Robbins dataset (stored under /catalogues)
craters = igen.ReadRobbinsCraterCSV(filename="catalogues/RobbinsLunarCraters.csv")
# craters = igen.ReadLROCHeadCombinedCraterCSV(filelroc="catalogues/LROCCraters.csv",
#                                   filehead="catalogues/HeadCraters.csv")
craters = igen.ResampleCraters(craters, source_cdim, con_img.size[1]) #crop relevant part
craters.tail()

In [ ]:
fn = '../data/test_SLDEM4'
sub_cdim = [0., 20., 0., 30.]

# If True, script prints out the image it's currently working on.
verbose = True

# Sample subset of image.  Co-opt igen.ResampleCraters to remove all
# craters beyond cdim (either sub or source).
if sub_cdim != source_cdim:
    img = igen.InitialImageCut(con_img, source_cdim, sub_cdim)

craters = igen.ResampleCraters(craters, sub_cdim, img.size[1])

start_time = time.time()

# Generate input images: 
igen.GenDataset(img, craters, fn, 
                 cdim=sub_cdim, amt=10, rawlen_range=[256, 2000], seed=1337, verbos)

elapsed_time = time.time() - start_time
if verbose:
    print("Time elapsed: {0:.1f} min".format(elapsed_time / 60.))

In [ ]:
fn = '../data/test_SLDEM3'
sub_cdim = [0., 20., 0., 30.]

gen_imgs = h5py.File(fn + '_images.hdf5', 'r')
sample_data = {'imgs': [gen_imgs['input_images'][...].astype('float32'),
                        gen_imgs['target_masks'][...].astype('float32')]}

# See 5 images of the set we produced: 
show_subset = np.random.choice(range(len(gen_imgs['cll_xy'])),size=10, replace=False)
fig = plt.figure(figsize=[16, 8])
# dem_map = plt.imshow(img, extent=sub_cdim, aspect="equal", cmap="YlGn")
plt.xlabel("Longitude"); plt.ylabel("Latitude")
for i in show_subset:
  llon,rlon,llat,ulat=gen_imgs['longlat_bounds']['img_{:02d}'.format(i)][...]
  h1,=plt.plot([llon, llon, rlon, rlon, llon],[llat, ulat, ulat, llat, llat],'r')
  plt.text(llon,llat,str(i), color='k')

# plt.colorbar()
    
fig2 = plt.figure(figsize=[16, 8])
for i, im_num in enumerate(show_subset):
  ax=plt.subplot(len(show_subset)//5,5,i+1)
  utils.maplot.plot_map_w_scalebar2(ax,gen_imgs,im_num)

# 2. Test model on frames from new DEM

In [ ]:
import keras
keras.__version__

In [ ]:
from keras.models import load_model
import sys
deepmoon_path='../DeepCrater'
sys.path.append("%s/utils/"%deepmoon_path)
import template_match_target as tmt

# Use the commented out line for Keras 2.0+
# model = load_model(zenodo_path + '/model_keras2.h5')
model = load_model(data_path + '/model_keras1.2.2.h5')

In [ ]:
fn = '../data/test_SLDEM4'
gen_imgs = h5py.File(fn + '_images.hdf5', 'r')
sample_data = {'imgs': [gen_imgs['input_images'][...].astype('float32'),
                        gen_imgs['target_masks'][...].astype('float32')]}
proc.preprocess(sample_data)
sd_input_images = sample_data['imgs'][0]
sd_target_masks = sample_data['imgs'][1]

In [ ]:
# Use the CNN to make a prediction.  We use [iwant:iwant + 1] to keep input as a 4D tensor.
# Input data MUST BE PREPROCESSED (see proc.preprocess, above) before being passed into model.predict
iwant = show_subset[2]
pred = model.predict(sd_input_images[iwant:iwant + 1])
# Use scikit-image template matching to extract crater locations.  Only search for craters with r >= 3 pixels.
extracted_rings = tmt.template_match_t(pred[0].copy(), minrad=1.)
fig = plt.figure(figsize=[18, 6])
[ax1, ax2, ax3] = fig.subplots(1, 3)
ax1.imshow(sd_input_images[iwant].squeeze(), origin='upper', cmap='YlGn')
ax1.imshow(1-sd_target_masks[iwant].squeeze(), origin='upper', cmap='Greys_r', alpha=0.3)
ax2.imshow(pred[0], origin='upper', cmap='Greys_r', vmin=0, vmax=1)
ax3.imshow(sd_input_images[iwant].squeeze(), origin='upper', cmap="YlGn")
for x, y, r in extracted_rings:
    circle = plt.Circle((x, y), r, color='blue', fill=False, linewidth=2, alpha=0.9)
    ax3.add_artist(circle)
ax1.set_title('DEM + Ground-Truth')
ax2.set_title('CNN Predictions')
ax3.set_title('Post-CNN Craters')
plt.show()

In [ ]:
import pandas as pd
df=pd.DataFrame(data=extracted_rings, columns=['x','y','r'])
print(df)
df.pivot_table(index='r', aggfunc='count')

In [ ]:
img=gen_imgs['input_images'][4]
# print(type(img), img.size, img.shape)
np.unique(img.flatten())